# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

In [22]:
install.packages("tokenizers")
install.packages("stringr")
install.packages("httr")
install.packages("jsonlite")
install.packages("digest")
install.packages("viridis")

library(tokenizers)
library(stringr)
library(httr)
library(jsonlite)
library(digest)
library(viridis)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)



#### a) Make a function to tokenize the text.

In [21]:
tokenize_text <- function(text) {
  tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}

#### b) Make a function generate keys for ngrams.

In [3]:
key_from <- function(ngram, sep = "\x1f") {
  paste(ngram, collapse = sep)
}

#### c) Make a function to build an ngram table.

In [4]:
build_ngram_table <- function(tokens, n, sep = "\x1f") {
  if (length(tokens) < n) return(new.env(parent = emptyenv()))
  tbl <- new.env(parent = emptyenv())
  
  for (i in seq_len(length(tokens) - n + 1L)) {
    ngram <- tokens[i:(i + n - 2L)]
    next_word <- tokens[i + n - 1L]
    key <- paste(ngram, collapse = sep)
    counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (next_word %in% names(counts)) {
      counts[[next_word]] <- counts[[next_word]] + 1L
    } else {
      counts[[next_word]] <- 1L
    }
    tbl[[key]] <- counts
  }
  tbl
}

#### d) Function to digest the text.

In [5]:
digest_text <- function(text, n) {
  tokens <- tokenize_text(text)
  build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [6]:
digest_url <- function(url, n) {
  res <- httr::GET(url)
  txt <- httr::content(res, as = "text", encoding = "UTF-8")
  digest_text(txt, n)
}

#### f) Function that gives random start.

In [7]:
random_start <- function(tbl, sep = "\x1f") {
  keys <- ls(envir = tbl, all.names = TRUE)
  if (length(keys) == 0) stop("No n-grams available. Digest text first.")
  picked <- sample(keys, 1)
  strsplit(picked, sep, fixed = TRUE)[[1]]
}

#### g) Function to predict the next word.

In [8]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
  key <- paste(ngram, collapse = sep)
  counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
  if (length(counts) == 0) return(NA_character_)
  sample(names(counts), size = 1, prob = as.numeric(counts))
}

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [23]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
  force(tbl); n <- as.integer(n); force(sep)
  function(start_words = NULL, length = 10L) {
    # if user does not give a start word, use random start
    if (is.null(start_words) || length(start_words) != n - 1L) {
      start_words <- random_start(tbl, sep = sep)
    }
    word_sequence <- start_words
    for (i in seq_len(max(0L, length - length(start_words)))) {
      ngram <- tail(word_sequence, n - 1L)
      next_word <- predict_next_word(tbl, ngram, sep = sep)
      if (is.na(next_word)) break
      word_sequence <- c(word_sequence, next_word)
    }
    paste(word_sequence, collapse = " ")
  }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [11]:
set.seed(2025)
url_grimm <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"  # Grimm’s Fairy Tales
tbl_grimm <- digest_url(url_grimm, n = 3)
gen_grimm <- make_ngram_generator(tbl_grimm, n = 3)

In [12]:
cat("Grimm's Fairy Tales — Start words 'the king':\n")
output_grimm_king <- gen_grimm(start_words = c("the", "king"), length = 15)
print(output_grimm_king)

Grimm's Fairy Tales — Start words 'the king':
[1] "the king has forbidden me to marry another husband am not i shall ride upon"


In [13]:
cat("\nGrimm's Fairy Tales — Random start:\n")
output_grimm_random <- gen_grimm(length = 15)
print(output_grimm_random)



Grimm's Fairy Tales — Random start:
[1] "song was over the lake and herself into her little daughter’s hand and was about"


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [14]:
url_armour <- "https://www.gutenberg.org/cache/epub/42322/pg42322.txt"  # Ancient Armour and Weapons in Europe
tbl_armour <- digest_url(url_armour, n = 3)
gen_armour <- make_ngram_generator(tbl_armour, n = 3)

In [15]:
cat("\nAncient Armour — Start words 'the king':\n")
output_armour_king <- gen_armour(start_words = c("the", "king"), length = 15)
print(output_armour_king)


Ancient Armour — Start words 'the king':
[1] "the king of vagabonds the mighty nimrod of sacred story and for moving with facility"


In [16]:
cat("\nAncient Armour — Random start:\n")
output_armour_random <- gen_armour(length = 15)
print(output_armour_random)


Ancient Armour — Random start:
[1] "request was made by congress for his edification the surrounding plain i learned upon inquiry"


#### c) Explain in 1-2 sentences the difference in content generated from each source.

The Grimm’s Fairy Tales model generated short, narrative phrases such as “the king has forbidden me to marry another husband…” and “song was over the lake…”, which reflect story-like and character-driven language typical of fairy tales. In contrast, the Ancient Armour model produced phrases like “the king of France it is…” and “regular gradations the first time amid the untrod wastes of humanity…”, which sound more historical, descriptive, and formal. This difference shows how the model captures the distinct vocabulary and tone of each text

## Question 3
#### a) What is a language learning model? 
#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

A) A language learning model is a type of machine learning model that predicts the probability of a sequence of words to understand and generate human language. It learns patterns from large amounts of text data and can then generate, summarize, or respond to text input by predicting the next most likely word/token. Examples include models like ChatGPT or Gemma.

B) If the internet goes down, you can run a language model locally using OLLAMA, a tool that lets you download and run models directly on your computer's terminal. You would use the following commands to install and run the model: brew install ollama, ollama -v, ollama pull gemma3:1b, ollama serve.

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | A system that uses the environment to parse command line arguments (command mkdir), handle signals, read from standard input, and write to standard output. |
| **Terminal emulator** | The application you're typing in that hosts the shell and provides the interface (ie: the terminal on a Mac) |
| **Process** | An environment that communicates using standard input/output |
| **Signal** |  A message sent to a process to control it  |
| **Standard input** | The default way a process receives data (read characters from the input) |
| **Standard output** | Normal output from a process (write characters to the output) |
| **Command line argument** |  The information we pass to a process when we start it (ie. Project) |
| **The environment** | All the stuff a process can see when it is running |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
#### b) Explain what this command is doing, part by part.

A) The programs used in this command are: find, xargs, grep

B) find . -iname "*.R" | xargs grep read_csv: in this statement "find" searches for files in a specified location. "." refers to the current working directory where the search will occur. "-iname" tells the search to look for files that match the "*.R" pattern. "xargs" takes the list of files produced by the "find" command and passes them as arguments to "grep", a patten matching program, which searches through text for the "read_csv" expression/string. Together, this command allows us to search each file with grep by using zargs to first find a set of files and then search for them.


## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.
#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.
#### c) How do you log in to the RStudio server?

A)
Hello from Docker!
This message shows that your installation appears to be working correctly.
 To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

 For more examples and ideas, visit:
 https://docs.docker.com/get-started/

B)
command: docker run -d -p 8787:8787 -e PASSWORD=yourpassword rocker/rstudio
output: 
bellawilliams@Bellas-MacBook-Air-6 ~ % docker run -d -p 8787:8787 -e PASSWORD=yourpassword rocker/rstudio
Unable to find image 'rocker/rstudio:latest' locally
latest: Pulling from rocker/rstudio
3665120d345d: Pulling fs layer 
b8a35db46e38: Pulling fs layer 
2c9ba66d5dbe: Pulling fs layer 
2034506aa72f: Pulling fs layer 
bcce866b1806: Pulling fs layer 
91ed5b86de88: Pulling fs layer 
9c1a4a0706b7: Pulling fs layer 
e2804bef35e8: Pulling fs layer 
a730ff463d58: Pull complete 
cc9c938c1f51: Pull complete 
bc9245ceaac5: Pull complete 
39038e16d1ba: Pull complete 
191985778909: Pull complete 
5b219f62ce36: Pull complete 
abd0190d83fb: Pull complete 
08e74fd5985d: Pull complete 
664fb1818bbb: Pull complete 
5d246ec925db: Pull complete 
Digest: sha256:9f85211a666fb426081a6f5a01f9f9f51655262258419fa21e0ce38a5afc78d8
Status: Downloaded newer image for rocker/rstudio:latest
639938fde6824ba384d2a311b8e2a7e5c33d04afbeb6c34972864d23e059e94f
bellawilliams@Bellas-MacBook-Air-6 ~ %


C) go to http://localhost:8787 in browser
login using username: rstudio password: yourpassword